# 2.3.1 概念

.requires_grad  
.backward()  
.grad  

.detach()  
with torch.no_grad()  

.grad_fn->Function

In [1]:
import torch
x=torch.ones(2,2,requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
y=x+2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


x是直接创建的，称为__叶子节点__，所以它没有grad_fn, 而y是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn。

In [3]:
z=y*y*3
out=z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


通过.requires_grad_()来用in-place的方式改变requires_grad属性

In [4]:
a=torch.randn(2,2)
a=((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)

False
True


# 2.3.3 梯度

因为out为标量，调用backward()无需指定求导变量

In [5]:
out.backward()

In [6]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


grad在反向传播的过程中时累加的

In [7]:
out2 = x.sum()
out2.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])


In [9]:
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


__不允许张量对张量求导，只允许标量对标量求导__，，求导结果和自变量同形的张量  
必要时我们要把张量通过所有张量的元素求和的方式转换为标量

In [13]:
x=torch.tensor([1.0,2.0,3.0,4.0],requires_grad=True)
y=2*x
z=y.view(2,2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [14]:
v=torch.tensor([[1.0,0.1],[0.01,0.001]])
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [15]:
x=torch.tensor(1.0,requires_grad=True)
y1=x ** 2
with torch.no_grad():
    y2=x **3
y3=y1+y2
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [16]:
y3.backward()
print(x.grad)

tensor(2.)


y2.grad_fn为false -> 与y2有关的梯度不会回传

In [17]:
y2.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

修改Tensor的数值，又不想被autograd记录，可以对__tensor.data__进行操作

In [29]:
x=torch.ones(1,requires_grad=True)
print(x.data)
print(x.data.requires_grad)

y=2*x
x.data*=100

y.backward()
print(x)
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
